In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from Stargazer1.stargazer import Stargazer # sima Stargazer csomagot előssör le kell tölteni, a Stargazer1-ben csak átírtam magyarra a változók nevét az eredménytáblákhoz
import matplotlib.pyplot as plt

In [ ]:
helicopter = pd.read_excel('helicopter.xlsx', sheet_name='goodcountries_nooutliers') # elérési út megadása, az outlieres verzióhoz sheet_name='goodcountries_justvalues'
helicopter['összes eset (1000 főre jutó)']=helicopter['összes eset']/helicopter['népesség']*1000
helicopter['összes halál (1000 főre jutó)']=helicopter['összes halál']/helicopter['népesség']*1000

In [ ]:
# Szűrt df, ha külön kell valamelyik minta (a negyedéves GDP/infláció, jegybanki alapkamat kontrollra is elérhető, Eurozóna nélküli stb.
# Alaphelyzetben kikapcsolva, #

#helicopter = helicopter[helicopter['negyedéves GDP kontroll'].notna()]
#helicopter = helicopter[helicopter['negyedéves infláció kontroll'].notna()]
#helicopter = helicopter[helicopter['jegybanki alapkamat változás'].notna()]
#helicopter = helicopter[helicopter['Eurozone']=='no']

In [ ]:
# A gdp_model, infl_model elemei, CSERÉLGETNI KELL A TAGOKAT AZ ADOTT BECSLÉSEK FÜGGVÉNYÉBEN! MINDIG AZOK A VÁLTOZÓK LEGYENEK ITT MEGADVA A MODELLBEN, AMIRE KÍVÁNCSIÁK VAGYUNK!
# Ha negyedéves GDP kontrollal megy a becslés, akkor '2019 reál GDP növekedés' és 'negyedéves GDP kontroll' a kontroll.
# Ha negyedéves GDP kontroll nélküli, akkor 2020 esetében csak '2019 reál GDP növekedés', 2021-től '2020 reál GDP növekedés' is kontroll.
# Ha van negyedéves inflációs kontroll, akkor 'negyedéves infláció kontroll' is bekerül a modellbe.
# A regressziós startgazer output táblában a .covariate_order mindig a megfelelő modell elemeit kell, hogy tartalmazza (első 3 változó ugyanaz mindig, utána a gdp_model3/infl_model3 változói, majd const).

heli_y = helicopter[['GDP 2020', 'reál GDP növekedés (1 év)', 'reál GDP növekedés (2 év)', 'reál GDP növekedés (3 év)', 'infláció 2020', 'infláció (1 év)', 'infláció (2 év)', 'infláció (3 év)']]
gdp_model1 = helicopter[['2019 reál GDP növekedés' ,'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)' ]]
gdp_model2 = helicopter[['2019 reál GDP növekedés', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya']]
gdp_model3 = helicopter[['2019 reál GDP növekedés','szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás']]
infl_model1 = helicopter[['átlagos infláció (2009-2019)','szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)']]
infl_model2 = helicopter[['átlagos infláció (2009-2019)', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya']]
infl_model3 = helicopter[['átlagos infláció (2009-2019)', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás']]
helicopter.iloc[:, 3:] = helicopter.iloc[:, 3:].apply(lambda x:pd.to_numeric(x, errors='coerce'))

In [ ]:
# GDP BECSLÉS

# BRUTTÓ ADÓSSÁG, MÉRLEGFŐÖSSZEG

#gdp_year változó, 2020-nál 0, 1 éves növekedésnél 1, 2 éves növekedésnél 2, 3 éves növekedésnél 3

gdp_year = 0

res_1 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (bruttó adósság, mérlegfőösszeg)']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (bruttó adósság, mérlegfőösszeg)']] ,gdp_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (bruttó adósság, mérlegfőösszeg)']] ,gdp_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4]) #Stargazer package kell hozzá
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['bruttó adósság', 'jegybank mérlegfőösszeg', 'helikopter interakció (bruttó adósság, mérlegfőösszeg)', '2019 reál GDP növekedés',  'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex()) #LaTeX kód, ami utána beolvaható
# Pythonban a summary-vel megnézhető az eredmény

In [ ]:
# GDP BECSLÉS

# BRUTTÓ ADÓSSÁG, KÖVETELÉSEK

gdp_year = 0

res_1 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (bruttó adósság, követelések)']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (bruttó adósság, követelések)']] ,gdp_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (bruttó adósság, követelések)']] ,gdp_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['bruttó adósság', 'kormányzattal szembeni bruttó követelés', 'helikopter interakció (bruttó adósság, követelések)', '2019 reál GDP növekedés', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

# Fiskális politika marginális hatásához az ábra

betas = res_2.params[1:4]
yaxes_fis = betas[0]+betas[2]*(helicopter['kormányzattal szembeni bruttó követelés']-np.nanmean(helicopter['kormányzattal szembeni bruttó követelés']))
xaxes_fis = (helicopter[['kormányzattal szembeni bruttó követelés']]-np.nanmean(helicopter[['kormányzattal szembeni bruttó követelés']]))/(np.nanvar(helicopter[['kormányzattal szembeni bruttó követelés']])**0.5)

plot=plt.plot(xaxes_fis,yaxes_fis)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Monetáris politika reakciója')
plt.ylabel('Fiskális politika marginális hatása')
plt.title('GDP')

In [ ]:
# GDP BECSLÉS

# KIADÁSOK, MÉRLEGFŐÖSSZEG

gdp_year = 0

res_1 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (kiadások, mérlegfőösszeg)']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (kiadások, mérlegfőösszeg)']] ,gdp_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (kiadások, mérlegfőösszeg)']] ,gdp_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['kormányzati kiadások', 'jegybank mérlegfőösszeg', 'helikopter interakció (kiadások, mérlegfőösszeg)', '2019 reál GDP növekedés', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# GDP BECSLÉS

# KIADÁSOK, KÖVETELÉSEK

#helicopter['helikopter interakció (kiadások, követelések)']=helicopter['kormányzati kiadások']*helicopter['kormányzattal szembeni bruttó követelés'] #ha nem centrálva szeretnénk az interakciós tagot megkapni

gdp_year = 3

res_1 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (kiadások, követelések)']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (kiadások, követelések)']] ,gdp_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (kiadások, követelések)']] ,gdp_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['kormányzati kiadások', 'kormányzattal szembeni bruttó követelés', 'helikopter interakció (kiadások, követelések)', '2019 reál GDP növekedés', 'negyedéves GDP kontroll', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# NEM HASZNÁLTAM VÉGÜL A DOLGOZATBAN

# GDP BECSLÉS

# IMF KIADÁSOK, MÉRLEGFŐÖSSZEG


gdp_year = 2

res_1 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (IMF, mérlegfőösszeg)']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (IMF, mérlegfőösszeg)']] ,gdp_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (IMF, mérlegfőösszeg)']] ,gdp_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['IMF fiskális kiadások (összes)', 'jegybank mérlegfőösszeg', 'helikopter interakció (IMF, mérlegfőösszeg)', '2019 reál GDP növekedés', 'negyedéves GDP kontroll', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# NEM HASZNÁLTAM VÉGÜL A DOLGOZATBAN

# GDP BECSLÉS

# IMF KIADÁSOK, KÖVETELÉSEK

gdp_year = 0

res_1 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (IMF, követelések)']] ,gdp_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (IMF, követelések)']] ,gdp_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,gdp_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (IMF, követelések)']] ,gdp_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['IMF fiskális kiadások (összes)', 'kormányzattal szembeni bruttó követelés', 'helikopter interakció (IMF, követelések)', '2019 reál GDP növekedés', 'negyedéves GDP kontroll', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# INFLÁCIÓ BECSLÉSE

# 2020: infl_year = 4, 1 év = 5, 2 év = 6, 3 év = 7

# BRUTTÓ ADÓSSÁG, MÉRLEGFŐÖSSZEG

infl_year = 4

res_1 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (bruttó adósság, mérlegfőösszeg)']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (bruttó adósság, mérlegfőösszeg)']] ,infl_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (bruttó adósság, mérlegfőösszeg)']] ,infl_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['bruttó adósság', 'jegybank mérlegfőösszeg', 'helikopter interakció (bruttó adósság, mérlegfőösszeg)', 'átlagos infláció (2009-2019)', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# INFLÁCIÓ BECSLÉSE

# BRUTTÓ ADÓSSÁG, KÖVETELÉSEK

infl_year = 9

res_1 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (bruttó adósság, követelések)']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (bruttó adósság, követelések)']] ,infl_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['bruttó adósság']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (bruttó adósság, követelések)']] ,infl_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['bruttó adósság', 'kormányzattal szembeni bruttó követelés', 'helikopter interakció (bruttó adósság, követelések)', 'átlagos infláció (2009-2019)', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# INFLÁCIÓ BECSLÉSE

# KIADÁSOK, MÉRLEGFŐÖSSZEG

infl_year = 5

res_1 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (kiadások, mérlegfőösszeg)']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (kiadások, mérlegfőösszeg)']] ,infl_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (kiadások, mérlegfőösszeg)']] ,infl_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['kormányzati kiadások', 'jegybank mérlegfőösszeg', 'helikopter interakció (kiadások, mérlegfőösszeg)', 'átlagos infláció (2009-2019)', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# INFLÁCIÓ BECSLÉSE

# KIADÁSOK, KÖVETELÉSEK

infl_year = 5

res_1 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (kiadások, követelések)']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (kiadások, követelések)']] ,infl_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['kormányzati kiadások']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (kiadások, követelések)']] ,infl_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['kormányzati kiadások', 'kormányzattal szembeni bruttó követelés', 'helikopter interakció (kiadások, követelések)', 'átlagos infláció (2009-2019)', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# NEM HASZNÁLTAM VÉGÜL A DOLGOZATBAN

# INFLÁCIÓ BECSLÉSE

# IMF, MÉRLEGFŐÖSSZEG

infl_year = 5

res_1 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (IMF, mérlegfőösszeg)']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (IMF, mérlegfőösszeg)']] ,infl_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['jegybank mérlegfőösszeg']], helicopter[['helikopter interakció (IMF, mérlegfőösszeg)']] ,infl_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['IMF fiskális kiadások (összes)', 'jegybank mérlegfőösszeg', 'helikopter interakció (IMF, mérlegfőösszeg)', 'átlagos infláció (2009-2019)', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())

In [ ]:
# NEM HASZNÁLTAM VÉGÜL A DOLGOZATBAN

# INFLÁCIÓ BECSLÉSE

# IMF, KÖVETELÉSEK

infl_year = 7

res_1 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_2 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (IMF, követelések)']] ,infl_model1], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_3 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (IMF, követelések)']] ,infl_model2], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
res_4 = sm.OLS(heli_y.iloc[:,infl_year], sm.add_constant(pd.concat([helicopter[['IMF fiskális kiadások (összes)']], helicopter[['kormányzattal szembeni bruttó követelés']], helicopter[['helikopter interakció (IMF, követelések)']] ,infl_model3], axis=1)), missing='drop').fit(cov_type='HC1', use_t=True)
stargazer = Stargazer([res_1, res_2, res_3, res_4])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'const': 'konstans'})
stargazer.covariate_order(['IMF fiskális kiadások (összes)', 'kormányzattal szembeni bruttó követelés', 'helikopter interakció (IMF, követelések)', 'átlagos infláció (2009-2019)', 'negyedéves GDP kontroll', 'szolgáltatások (GDP százalékában)', 'Covid lezárás index (stringency)', 'összes halál (1000 főre jutó)', '65 évnél idősebbek aránya', 'nettó energiaimport', 'kormányzati hatékonyság index', 'jegybanki alapkamat változás', 'const'])
print(stargazer.render_latex())